# This is a final Coursera IBM Data Science project

### Business Probmlem
Currently, it is not many places where one could buy a Gelato in Prague, but potential demand is high given the long queues.

### Instrumentation
In this capstone project, I will leverage the FourSquare API and K-means algorithm to find the best place to open a Gelato shop.
To visualize the result of my work I will use Folium.

### Target Audience
Businessmen who are interested in high margin business in Prague

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
